In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime, timezone
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

## ============ Phase 1 ====================================================

#### Get a Playlist's Items (song_id and duration)

In [2]:
def get_ID(user_id, token, playlist_id, market = 'US'):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

#### Get (audio) information about a song

In [3]:
def get_song(user_id, token, song_id, market = 'US'):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    response = requests.get(query, 
                   headers={"Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    album_id = json_response['album']['id'] if 'album' in json_response.keys() else None
    artist_id = json_response['artists'][0]['id'] if 'artists' in json_response.keys() else None
    
    ## get genre of the album
    base_url = "https://api.spotify.com/v1/albums/"
    query = f'{base_url}{album_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    genre = json_response['genres'] if 'genres' in json_response.keys() else None
    
    
    df_response['artist_id'] = artist_id if artist_id is not None and len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if album_id is not None and len(album_id) >0 else None
    df_response['genre'] = genre if genre is not None and len(genre) >0 else None

    return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')

#### Get Recommendations for a song using Spotify API

In [4]:
def check(par, par_range):
    ''' Check if values are in the feasible range. '''
    return(np.clip(par, par_range[0], par_range[1]))

def check_whole(kwargs):
    ''' Run the above check function to all optional parameters. '''
    pars = ['valence', 'speechiness', 'acousticness', 'liveness', 'danceability', 'hahaha']
    ranges = {'valence': [0,1], 'speechiness': [0,1], 'acousticness': [0,1], 'liveness': [0,1], 'danceability': [0,1]}
    if len(kwargs) == 0:
        return(kwargs)
    else:
        for item in pars:
            if np.array([re.match(f"(.*?)({item})", x) is not None for x in kwargs.keys()]).any(): 
                item_min = 'min_' + item
                item_max = 'max_' + item
                item_target = 'target_' + item
                par_range = ranges.get(item)
                kwargs[item_target] = check(kwargs.get(item_target), par_range)
                kwargs[item_min] = check(kwargs.get(item_min), par_range)
                kwargs[item_max] = check(kwargs.get(item_max), par_range)
        return(kwargs)
    
def recommend(song, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
    seed_tracks = song.index[0]
    
    time_mt = require_percents.get('time')
    energy_percent = require_percents.get('energy')
    instrumentalness_percent = require_percents.get('instrumentalness')
    tempo_percent = require_percents.get('tempo')
    
    min_duration_ms = int(max(target_duration_ms - time_mt * 1000, 0))
    max_duration_ms = int(target_duration_ms + time_mt * 1000)
    
    ## required percents
    target_energy = song['energy'][0]
    min_energy = check(target_energy * (1 - energy_percent), [0,1])
    max_energy = check(target_energy * (1 + energy_percent), [0,1])
    target_instrumentalness = song['instrumentalness'][0]
    min_instrumentalness = check(target_instrumentalness * (1 - instrumentalness_percent), [0,1])
    #max_instrumentalness = check(target_instrumentalness * (1 + instrumentalness_percent), [0,1])
    target_tempo = song['tempo'][0]
    min_tempo = check(target_tempo * (1 - tempo_percent), [0, np.inf])
    max_tempo = check(target_tempo * (1 + tempo_percent), [0, np.inf])

    ## default parameters
    target_key = song['key'][0]
    target_danceability=song['danceability'][0]
    target_mode=song['mode'][0]
         
    ## optional percents
    for item in optional_percents: 
        item_min = 'min_' + item
        item_max = 'max_' + item
        item_target = 'target_' + item
        target = song[item][0]
        kwargs[item_target] = target
        if item not in ['mode', 'key', 'popularity', 'time_signature']:
            kwargs[item_min] = target * (1 - optional_percents.get(item))
            kwargs[item_max] = target * (1 + optional_percents.get(item))
    kwargs.pop('optional_percents', None)   ## remove `optional_percents` from kwargs
    kwargs = check_whole(kwargs)
    
    endpoint_url = "https://api.spotify.com/v1/recommendations?"
    query = f'{endpoint_url}limit={limit}&seed_tracks={seed_tracks}'
    query += f'&target_duration_ms={target_duration_ms}&min_duration_ms={min_duration_ms}&max_duration_ms={max_duration_ms}'
    query += f'&target_energy={target_energy}&min_energy={min_energy}&max_energy={max_energy}'
    query += f'&target_instrumentalness={target_instrumentalness}&min_instrumentalness={min_instrumentalness}'
    query += f'&target_tempo={target_tempo}&min_tempo={min_tempo}&max_tempo={max_tempo}'
    query += f'&target_key={target_key}'
    query += f'&target_danceability={target_danceability}'
    query += f'&target_mode={target_mode}'
    
    if song['genre'][0] is not None:
        seed_genres = song['genre'][0]
        query += f'&seed_genres={seed_genres}'
    if song['artist_id'][0] is not None:
        seed_artist = song['artist_id'][0]
        query += f'&seed_artist={seed_artist}'

    if len(kwargs.keys()) > 0:
        query += '&'
        lst = [str(x[0])+'='+str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all limits from kwargs
    uris = [] 
    #print(query)
    
    response = requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    json_response = response.json()
    #print('Recommended Songs:')
    uris = []
    for i,j in enumerate(json_response['tracks']):
        uris.append(j['uri'])
        #print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}")
    return uris

#### Filter recommended songs based on the trend of energy

In [5]:
def filter_energy(r, pre_energy, market, trend = None):
    ''' filter recommended songs according to the energy trend '''
    
    if trend is None or pre_energy == 0:
        return r[0]
    
    energys_diff = [get_song(user_id, token, r[i][14:], market)['energy'][0] - pre_energy for i in range(len(r))]
    if trend == '+':
        if all(np.array(energys_diff) <= 0):
            return None
        idx = np.argmax( np.array(energys_diff) > 0)
    else:
        if all(np.array(energys_diff) >= 0):
            return None
        idx = np.argmax( np.array(energys_diff) < 0)
    return(r[idx])

#### Create a playlist

In [6]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public = False):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

#### Create bucket

In [7]:
def create_bucket(playlist_id, bucket_lengths = None):
    ids, durations = get_ID(user_id, token, playlist_id)
    
    if bucket_lengths is None: ## default: the whole playlist is a bucket
        end_songs = len(durations)-1
    elif (type(bucket_lengths) == 'int'): ## buckets with the same length
        bucket_lens = [bucket_lengths * 60 * 1000 for i in range(np.sum(durations) // bucket_lengths + 1)]
    elif (len(bucket_lengths) == 1): ## buckets with the same length
        bucket_lens = [bucket_lengths[0] * 60 * 1000 for i in range(np.sum(durations) // bucket_lengths[0] + 1)]
    else:
        bucket_lens = [x * 60 * 1000 for x in bucket_lengths]
    
    bucket = 0
    end_songs = [None] * len(bucket_lens)
    cur_length = 0
    for e, l in enumerate(durations):
        if np.abs(cur_length + l - bucket_lens[bucket]) < np.abs(cur_length - bucket_lens[bucket]):
            end_songs[bucket] = e
            cur_length += l
        else:
            bucket += 1
            if bucket == len(end_songs):
                break
            end_songs[bucket] = e
            cur_length += l - bucket_lens[bucket - 1]
         
    
    start = 0
    buckets = []
    for i in end_songs:
        buckets.append(ids[start:(i+1)])
        start = i+1
    return(buckets)

## ============ Phase 2 ====================================================

### Get a user's playlists (Nov 28 updated)

In [8]:
def get_plst_ID(user_id, token, market = 'US', added_time = False):
    tidy = lambda s: s[17:]
    
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    plst_id = [tidy(x['uri'])for x in json_response['items']]
    if not added_time:
        return plst_id
    else:
        base_url = "https://api.spotify.com/v1/playlists/"
        add = []
        for idx in plst_id:
            query = f'{base_url}{idx}/tracks?market={market}'
            response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
            json_response = response.json()
            add.append(json_response['items'][0]['added_at'])
        return(plst_id, add)

### Get information about a song (Dec 5 updated) 

In [9]:
def get_song_name(user_id, token, song_id, market = 'US'):
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    return df_response[['name', 'popularity', 'duration_ms', 'id']].set_index('id')

### Get a user's song frequency from playlists (Dec 5 updated)

In [10]:
def convert_weight(time_list, half_decay = 1):
    ''' Calculate weights.'''
    ## I use exponential decay now.
    ## the weight decays to 50% every `half_decay` days
    today = datetime.now(timezone.utc)
    log_weights = np.array([- (today - parse(s)).days - (today - parse(s)).seconds / 86400 for s in time_list])
    log_weights -= np.max(log_weights)
    return np.exp(log_weights / half_decay * np.log(2))

def get_song_freq(user_id, token, market, start_plst, end_plst, max_num = 100, use_time = True, decay_rate = 15):
    plst_ids, add = get_plst_ID(user_id, token, market, added_time = True)
    plst_ids, add = plst_ids[start_plst:(end_plst+1)], add[start_plst:(end_plst+1)]
    if use_time:
        weights = convert_weight(add, decay_rate)
    else:
        weights = [np.exp(-0.5*x) for x in range(len(add))]
    ids = [get_ID(user_id, token, playlist, market)[0] for playlist in plst_ids]
    song_list = [(s, w) for ls, w in list(zip(ids, weights)) for s in ls]
    counts = {}
    for s, w in song_list:
        counts[s] = counts.get(s, 0) + w
    df = pd.DataFrame(sorted(counts.items(), key = lambda x: -x[1]), columns = ['id', 'weighted_freq'])
    df = df.set_index('id').iloc[:max_num]
    dff = pd.concat([get_song_name(user_id, token, idx, market = 'US') for idx in df.index])
    return(pd.merge(df, dff, how = 'left', on = 'id')).reset_index().set_index('name')

### Get a user's common songs from playlist history (Dec 5 updated)

In [11]:
def count_songs(user_id, token, market):
    plst_ids= get_plst_ID(user_id, token, market)
    ids = [get_ID(user_id, token, playlist, market)[0] for playlist in plst_ids]
    return(len(ids))

In [12]:
def common(top, user_id, token, market, **kwargs):
    num_songs = count_songs(user_id, token, market)
    top = round(num_songs * top)
    kwargs.pop('max_num', None)        
    freq_df = get_song_freq(user_id, token, market, max_num = top, **kwargs)
    return(list(freq_df.iloc[:top].reset_index()['id']))

### Generate a similar playlist (Dec 8 updated) 

**If a common song is in the recommendation candidates, we will use it instead of filtering based on energy.**

In [13]:
def get_recommendation_for_a_playlist(user_id, token, name, description, limit, playlist_id=None, songs_id=None, create_pl=False, public=False, quiet=False, **kwargs):
    if not quiet:
        print("playlist", playlist_id)
    
    market = kwargs.get("market", 'US')
    kwargs["market"] = market
    
    common_top = kwargs.get("common_top", 0.25) ## default: songs that have a frequency within the top 25% are regarded as common songs 
    start_plst = kwargs.get("start_plst", 0)
    end_plst = kwargs.get("end_plst", 20)
    common_songs = ['spotify:track:'+ x for x in common(common_top, user_id, token, market = market, start_plst = start_plst, end_plst = end_plst)]
    #print(common_songs) 
    
    percent_args = kwargs.get("percent_args", {})
    requires = ['time', 'energy', 'instrumentalness', 'tempo']
    require_percents = {}
    optional_percents = {}
    for item in requires:  ## for required percents, extract them and store in `require_percents`
        require_percents[item] = percent_args.get(item, 0.1)
    for item in list(set(percent_args) - set(requires)): ## for optional percents, store them in `optional_percents`
        optional_percents[item] = percent_args.get(item)
    kwargs.pop('percent_args', None)        
    if not quiet:
        print("args:", kwargs, require_percents, optional_percents)
    
    def _get_one_recommendation(idx, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
        df_response = get_song(user_id, token, idx, market)
        r = recommend(df_response, limit, target_duration_ms, require_percents, optional_percents, **kwargs)
        if not quiet:
            print(f"For the {j+1}-th song, no recommendations, enlarged the range about duration.")
        new_time_percent = require_percents.get('time')
        new_energy_percent = require_percents.get('energy')
        new_require_percents = require_percents.copy()
        i = 0
        while len(r) == 0 and i < 10:
            i += 1
            new_time_percent += 10
            new_energy_percent *= 2
            new_require_percents['time'] = new_time_percent
            #new_require_percents['energy'] = new_energy_percent
            r = recommend(df_response, limit, target_duration_ms, new_require_percents, optional_percents, **kwargs)
        if i == 10:
            ## right now I keep the original one, we can improve it later
            r = ['spotify:track:' + df_response.index[0]]
            ## another way is to remove all optional limitations
            # r = recommend(df_response, limit, target_duration_ms, require_percents, {}, **kwargs)
        return r
        
    if playlist_id is not None:
        ## if playlist_id is specified, songs_id will not be used.
        ids, durations = get_ID(user_id, token, playlist_id, market)
    elif songs_id is not None:
        ids = songs_id
        durations = [get_song(user_id, token, ids[j], market)['duration_ms'][0] for j in range(len(ids))]
    else:
        NameError('Please input either playlist_id or songs_id.')
    
    common_num = round(kwargs.get("common_prop", 0) * len(ids)) ## default: don't use common song to substitute recommendations
    print(common_num)
    
    uris = []
    duration = 0
    diff = 0
    pre_energy = 0
    ene = 0
    trend = None
    current_common_num = 0
    candidate = []
    for j in range(len(ids)):
        energy = get_song(user_id, token, ids[j], market)['energy'][0] 
        trend = '+' if energy > pre_energy else '-'
        _limit = limit
        _r = None
        i = 0
        while _r is None and i < 10:
            ## if no songs satisfy requirements, we get more recommendations from API and then filter
            r = _get_one_recommendation(ids[j], _limit, durations[j]-diff, require_percents, optional_percents, **kwargs)    
            ######### if a common song exists in the recommendations, we will adopt it, rather than filtering based on energy trend
            if len(set.intersection(set(common_songs), set(r))) > 0 and common_num > current_common_num:
                _r = [i for i in common_songs if i in r][0]
                current_common_num += 1
                print('common song', _r[14:], 'is used.')
                break
            _r = filter_energy(r, ene, market, trend) ## add limitations to filter the recommendations and keep the energy trend
            if len(r) < _limit: ## no matter how to increase the limitation on the number of recommendations, result will no change
                break
            _limit = _limit + 10
            i = i+1
        candidate.append(r)
        if i == 10 or _r is None:
            _r = r[0] ## if still no songs satisfy requirements, use the most similar song from Spotify Recommendation API
        if not quiet:
            print('candidate: ', [x[14:] for x in r])
            print('recommendation: ', _r[14:])
        uris.append(_r)
        dur = get_song(user_id, token, _r[14:], market)['duration_ms'][0] ## duration of the recommended song
        duration += dur
        diff = duration - sum(durations[:j+1])
        ene = get_song(user_id, token, _r[14:], market)['energy'][0] ## energy of the recommended song
        pre_energy = energy  ## energy of the j-th song in the playlist

    if create_pl:
        create_plst(user_id, token, [x for x in uris if x is not None], name, description, public)
    
    used_common = list(filter(lambda x: x in uris, common_songs))
    return(uris, candidate, common_songs, used_common, common_num, current_common_num)

### Find place for common songs (Dec 8 updated) 

In [14]:
def place_common(user_id, token, market, common_songs, num, playlist_id=None, songs_id=None, pars_weight = None):
    if playlist_id is not None:
        ids, _ = get_ID(user_id, token, playlist_id, market)
    elif songs_id is not None:
        ids = songs_id
    else:
        NameError('Please input either playlist_id or songs_id.')
        
    pars = ['energy', 'duration_ms', 'instrumentalness', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo', 'time_signature']
    if pars_weight is None:
        cols = ['energy', 'duration_ms', 'instrumentalness']
        weight = [0.5, 0.4, 0.1] 
    else:
        cols = list(filter(lambda x: x in pars, pars_weight.keys()))
        weight = [pars_weight[key] for key in cols]
        weight = list(np.array(weight) / np.sum(weight))
    
    ids = [x[14:] for x in songs_id]
    common_songs = [x[14:] for x in common_songs]
    dff = pd.concat([get_song(user_id, token, idx, market) for idx in common_songs])._get_numeric_data()[cols]
    
    candidate = {key: (None, np.inf) for key in dff.index}
    for i in range(len(ids)):
        if dff.shape[0] < 1:
            break
        idx = ids[i]
        info = get_song(user_id, token, idx, market)[cols]
        diff = ((dff.sub(info._get_numeric_data().values[0].tolist(), axis='columns').applymap(np.square))*weight).sum(axis = 1).sort_values()
        for j in range(diff.shape[0]):
            if diff[j] < candidate[diff.index[j]][1]:
                candidate[diff.index[j]] = (i, diff[j])
    
    sorted_candidate = sorted(candidate.items(),key=lambda x: x[1][1])
    output = {key: values[0] for (key, values) in sorted_candidate}
    return(['spotify:track:' + x for x in list(output.keys())[:num]], list(output.values())[:num])

### Meet middle

In [15]:
def meet_middle(user_id, token, limit, songs_id, **kwargs):
    f_pl, _, _, _, _ = get_recommendation_for_a_playlist(user_id, token, name='', description='', limit=limit, songs_id=songs_id, **kwargs)
    b_pl, _, _, _, _  = get_recommendation_for_a_playlist(user_id, token, name='', description='', limit=limit, songs_id=songs_id[::-1], **kwargs)
    b_pl = b_pl[::-1]
    if f_pl == b_pl or len(f_pl) == 1:
        return(f_pl)
    else:
        middle = len(f_pl) // 2
        energys = [get_song(user_id, token, x)['energy'][0]  for x in songs_id]
        trends = [1 if energys[i+1] > energys[i] else -1 for i in range(len(energys)-1)]
        f_energy = get_song(user_id, token, f_pl[middle-1])['energy'][0] 
        b_energy = get_song(user_id, token, b_pl[middle])['energy'][0]
        trend = trends[middle-1]
        if f_pl[middle] == b_pl[middle] or trend * (b_energy - f_energy) > 0:
            return(f_pl[:middle] + b_pl[middle:])
        else:
            offset = 1
            finished = None
            while trend * (b_energy - f_energy) <= 0 and middle+1+offset < len(b_pl)-1:
                f_energy = get_song(user_id, token, f_pl[middle-1+offset])['energy'][0] 
                b_energy = get_song(user_id, token, b_pl[middle+1+offset])['energy'][0]
                trend = trends[middle-1+offset]
                if trend * (b_energy - f_energy) > 0:
                    return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                else:
                    offset += 1
        if finished is None:
            offset = -1
            while trend * (b_energy - f_energy) <= 0 and middle-1+offset > 0:
                f_energy = get_song(user_id, token, f_pl[middle-1+offset])['energy'][0] 
                b_energy = get_song(user_id, token, b_pl[middle+1+offset])['energy'][0]
                trend = trends[middle-1+offset]
                if trend * (b_energy - f_energy) > 0:
                    return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                else:
                    offset += -1
        if finished is None:
            ### if after searching from the middle to the end and the beginning, no points between 
            ### these two playlists can be combined, we just use the original one that goes from the beginning
            return (f_pl)

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [16]:
# settings
token = "BQDssGmKoWVgdj4jUtYCAaBfZ_yumzSaeCnP0M3C4wZg3WyQ2WLY8iyYBk2SyW7w06oysORfMY8iDvXQ4LOS3nMIlBraWx8Wf942Ej-whmnUVynYAuokbF8_CJL39K3oQO88PqO_8iddfcFiMRMPQxskPq3LxiRt9UjRYJW6Yn1Ppdz7TmsZeXl9hpvqUXKfyRQ7HLxWUKNirg"
user_id = "pbwppse1hilahmk43ls424ao4"

name = 'new method no energy 6/21'   ## name for the new playlist
description = 'new method' ## description for the new playlist
#f'recommendations based on {re.match("(.*?)Recommandation", name).group(1)}' 
limit = 10  ## for each song in current playlist, 10 recommendations are generated

plst_id = get_plst_ID(user_id, token)  ## get id for each playlist from a user
#playlist_id = plst_id[7]    ## set the 2nd playlist to be the target playlist
playlist_id = '4N2UHn9HpFc3n93s1gduIM'
percent_args = {'time': 15, 'energy': 0.1} #'liveness': 0.05

In [17]:
song_recom, candidate, common_songs, used_common, common_num, current_common_num= get_recommendation_for_a_playlist(user_id, token, name, description, limit, playlist_id, create_pl=False, market='US', 
                                  percent_args = percent_args, 
                                  common_top = 0.6, start_plst = 2, end_plst = 10, common_prop = 0.2, quiet = True)

3


In [18]:
song_recom

['spotify:track:7bzks4LGpQUuPKBzJ6iQ7y',
 'spotify:track:1qg2GMCuGX0T4VoFI5kOee',
 'spotify:track:3PAGiKZQwOwVM3Z80b4UAD',
 'spotify:track:4Uw7NtaXY0xJrbR9qiaN4H',
 'spotify:track:4GrcbD2aHVUvn0KvvnlKeR',
 'spotify:track:6GYD9MwygCfPurS4Dd7uvT',
 'spotify:track:7caJcFZTtLzy0ZSol1AXKH',
 'spotify:track:1tQnFdcE9VeqLyIHS55ijF',
 'spotify:track:0RrVVSeayZOynI2ukmYhpk',
 'spotify:track:2xql0pid3EUwW38AsywxhV',
 'spotify:track:0Sept02SvvMyFBka1KLcOL',
 'spotify:track:4Fknl5coWTUwhBHuABw5lu',
 'spotify:track:0tBCr4Xvvc3XyDYd0B4YsS']

In [19]:
candidate

[['spotify:track:7bzks4LGpQUuPKBzJ6iQ7y'],
 ['spotify:track:1qg2GMCuGX0T4VoFI5kOee',
  'spotify:track:7nBR4Tt431p1MTgv3lVsmX',
  'spotify:track:6AQ9EbL5EfYlrIX3s2gVa7',
  'spotify:track:1pJ4sLJuth2iKvHzFjU0bZ',
  'spotify:track:23hiSG0y0cRBa34mDWLp1y',
  'spotify:track:1Tda8qcRED3rn0nUDSY07F',
  'spotify:track:26fdI6wu2JzD1IPsRIedQV',
  'spotify:track:5ymor3b8Gs82TouTl7z7Yj',
  'spotify:track:6OEfLAUzFUe3xVdsXkU5eZ',
  'spotify:track:2Uf2oU5DifHnJD7rcQcBpM'],
 ['spotify:track:3PAGiKZQwOwVM3Z80b4UAD',
  'spotify:track:50L4TGtO5U2MktfsHDfXSA'],
 ['spotify:track:4Uw7NtaXY0xJrbR9qiaN4H'],
 ['spotify:track:4GrcbD2aHVUvn0KvvnlKeR',
  'spotify:track:6JU6B3alhgZNNBTsgfMDAp',
  'spotify:track:5860DNiiXsxmcRrZquqNH4'],
 ['spotify:track:6GYD9MwygCfPurS4Dd7uvT',
  'spotify:track:5aoJnOhycrs0NtXomySi3e'],
 ['spotify:track:7caJcFZTtLzy0ZSol1AXKH',
  'spotify:track:5hNeatT8kKThMjqGNi9SZk'],
 ['spotify:track:1tQnFdcE9VeqLyIHS55ijF',
  'spotify:track:4FmvlFR6xsp1VbFZdjhSmR',
  'spotify:track:3T0iZASqaz

In [20]:
common_songs

['spotify:track:0ZCRriwMzsMfBXQn2T47SR',
 'spotify:track:3ZTRTKx8uqpzqqWOA3JyNN',
 'spotify:track:2dnIR38rKEDmwbE2bk589B',
 'spotify:track:3FUBJQIMv98S3SNKWiDshn']

In [21]:
used_common

[]

In [22]:
current_common_num

0

### Test for place common songs
#### Change some recommendations to common songs

In [34]:
common_songs

['spotify:track:0ZCRriwMzsMfBXQn2T47SR',
 'spotify:track:3ZTRTKx8uqpzqqWOA3JyNN',
 'spotify:track:2dnIR38rKEDmwbE2bk589B',
 'spotify:track:3FUBJQIMv98S3SNKWiDshn']

In [35]:
## all possible parameters
## ['energy', 'duration_ms', 'instrumentalness', 'danceability', 'key', 'loudness', 
## 'mode', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo', 'time_signature']
   
pars_weight = {'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2}

In [36]:
songs = [x if x not in common_songs else None for x in song_recom]
songs_id = [x for x in songs if x is not None]
market = 'US'
com, place = place_common(user_id, token, market = market, 
                          common_songs = list(filter(lambda x: x not in used_common, common_songs)), 
                          num = common_num - current_common_num, songs_id = songs_id, 
                          pars_weight = pars_weight)

In [37]:
com, place

(['spotify:track:0ZCRriwMzsMfBXQn2T47SR',
  'spotify:track:3FUBJQIMv98S3SNKWiDshn',
  'spotify:track:3ZTRTKx8uqpzqqWOA3JyNN'],
 [8, 10, 6])

In [38]:
## replace recommendations with common songs
count = 0
for i in range(len(songs)):
    if songs[i] is not None:
        count +=1
        for j in range(len(place)):
            if count == place[j]:
                songs[i] = com[j]
    else:
        songs[i] = song_recom[i]

In [39]:
# old
song_recom

['spotify:track:7bzks4LGpQUuPKBzJ6iQ7y',
 'spotify:track:1qg2GMCuGX0T4VoFI5kOee',
 'spotify:track:3PAGiKZQwOwVM3Z80b4UAD',
 'spotify:track:4Uw7NtaXY0xJrbR9qiaN4H',
 'spotify:track:4GrcbD2aHVUvn0KvvnlKeR',
 'spotify:track:6GYD9MwygCfPurS4Dd7uvT',
 'spotify:track:7caJcFZTtLzy0ZSol1AXKH',
 'spotify:track:1tQnFdcE9VeqLyIHS55ijF',
 'spotify:track:0RrVVSeayZOynI2ukmYhpk',
 'spotify:track:2xql0pid3EUwW38AsywxhV',
 'spotify:track:0Sept02SvvMyFBka1KLcOL',
 'spotify:track:4Fknl5coWTUwhBHuABw5lu',
 'spotify:track:0tBCr4Xvvc3XyDYd0B4YsS']

In [40]:
# new
songs

['spotify:track:7bzks4LGpQUuPKBzJ6iQ7y',
 'spotify:track:1qg2GMCuGX0T4VoFI5kOee',
 'spotify:track:3PAGiKZQwOwVM3Z80b4UAD',
 'spotify:track:4Uw7NtaXY0xJrbR9qiaN4H',
 'spotify:track:4GrcbD2aHVUvn0KvvnlKeR',
 'spotify:track:3ZTRTKx8uqpzqqWOA3JyNN',
 'spotify:track:7caJcFZTtLzy0ZSol1AXKH',
 'spotify:track:0ZCRriwMzsMfBXQn2T47SR',
 'spotify:track:0RrVVSeayZOynI2ukmYhpk',
 'spotify:track:3FUBJQIMv98S3SNKWiDshn',
 'spotify:track:0Sept02SvvMyFBka1KLcOL',
 'spotify:track:4Fknl5coWTUwhBHuABw5lu',
 'spotify:track:0tBCr4Xvvc3XyDYd0B4YsS']

In [41]:
## we can check those common songs in the updated recommendations
list(filter(lambda x: x not in song_recom, songs))

['spotify:track:3ZTRTKx8uqpzqqWOA3JyNN',
 'spotify:track:0ZCRriwMzsMfBXQn2T47SR',
 'spotify:track:3FUBJQIMv98S3SNKWiDshn']

In [30]:
#### if you want to create a playlist with these recommendations, run it.
name = 'test for common songs'
description = 'with 4 common songs'
public = False
create_plst(user_id, token, [x for x in songs if x is not None], name, description, public)

Playlist test for common songs is successfully created!
Playlist test for common songs is successfully filled with recommendations!
Your playlist is ready at https://open.spotify.com/playlist/0sB10sCofKGpRefQOrjpvZ


### Test for song frequency

In [31]:
start_plst = 0 ## 0 means start from the most recent one
end_plst = 10 ## 10 means end at the 10 recent one
## in this case, 11 playlists are used 

#### Use create time for playlist

In [32]:
df = get_song_freq(user_id, token, market = 'US', start_plst = 0, end_plst = 20, use_time = False)
df.head(10)

,id,weighted_freq,popularity,duration_ms
name,,,,
Playground,7bzks4LGpQUuPKBzJ6iQ7y,1.974410,47,274386
Empty With You,7caJcFZTtLzy0ZSol1AXKH,1.974410,36,203520
"I Want To Hold Your Hand - From ""Across The Universe"" Soundtrack",4Fknl5coWTUwhBHuABw5lu,1.591010,45,165533
Underneath,0tBCr4Xvvc3XyDYd0B4YsS,1.591010,40,206883
I Feel Love,4Uw7NtaXY0xJrbR9qiaN4H,1.367879,37,422400
Rocket Fuel (feat. De La Soul),3ZTRTKx8uqpzqqWOA3JyNN,1.217420,56,195666
Oooh - Original Mix,0ZCRriwMzsMfBXQn2T47SR,1.217420,32,386065
The Truth that You Leave,3FUBJQIMv98S3SNKWiDshn,1.049787,38,223467
Love Core,1qg2GMCuGX0T4VoFI5kOee,1.000000,18,416558


#### Don't use create time for playlist

In [33]:
df1 = get_song_freq(user_id, token, market = 'US', start_plst = 0, end_plst = 20, use_time = True, decay_rate = 15)
df1.head(10)

,id,weighted_freq,popularity,duration_ms
name,,,,
Playground,7bzks4LGpQUuPKBzJ6iQ7y,1.611422,47,274386
Empty With You,7caJcFZTtLzy0ZSol1AXKH,1.611422,36,203520
"I Want To Hold Your Hand - From ""Across The Universe"" Soundtrack",4Fknl5coWTUwhBHuABw5lu,1.249484,45,165533
Underneath,0tBCr4Xvvc3XyDYd0B4YsS,1.249484,40,206883
I Feel Love,4Uw7NtaXY0xJrbR9qiaN4H,1.249146,37,422400
Rocket Fuel (feat. De La Soul),3ZTRTKx8uqpzqqWOA3JyNN,1.131339,56,195666
Oooh - Original Mix,0ZCRriwMzsMfBXQn2T47SR,1.131339,32,386065
The Truth that You Leave,3FUBJQIMv98S3SNKWiDshn,1.090227,38,223467
Love Core,1qg2GMCuGX0T4VoFI5kOee,1.000000,18,416558
